[View in Colaboratory](https://colab.research.google.com/github/vsolodkyi/folium/blob/master/DeepLearning.ipynb)

In [5]:
!pip install imageio
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

    100% |████████████████████████████████| 3.3MB 7.8MB/s 


In [22]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

last_percent_reported = None
data_root = '/' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Attempting to download: notMNIST_large.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified /notMNIST_large.tar.gz
Attempting to download: notMNIST_small.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified /notMNIST_small.tar.gz


In [24]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  print (data_folders)
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

/notMNIST_large already present - Skipping extraction of /notMNIST_large.tar.gz.
['/notMNIST_large/A', '/notMNIST_large/B', '/notMNIST_large/C', '/notMNIST_large/D', '/notMNIST_large/E', '/notMNIST_large/F', '/notMNIST_large/G', '/notMNIST_large/H', '/notMNIST_large/I', '/notMNIST_large/J']
['/notMNIST_large/A', '/notMNIST_large/B', '/notMNIST_large/C', '/notMNIST_large/D', '/notMNIST_large/E', '/notMNIST_large/F', '/notMNIST_large/G', '/notMNIST_large/H', '/notMNIST_large/I', '/notMNIST_large/J']
/notMNIST_small already present - Skipping extraction of /notMNIST_small.tar.gz.
['/notMNIST_small/A', '/notMNIST_small/B', '/notMNIST_small/C', '/notMNIST_small/D', '/notMNIST_small/E', '/notMNIST_small/F', '/notMNIST_small/G', '/notMNIST_small/H', '/notMNIST_small/I', '/notMNIST_small/J']
['/notMNIST_small/A', '/notMNIST_small/B', '/notMNIST_small/C', '/notMNIST_small/D', '/notMNIST_small/E', '/notMNIST_small/F', '/notMNIST_small/G', '/notMNIST_small/H', '/notMNIST_small/I', '/notMNIST_smal

# Problem 1
Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.
## New Section
Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road.

A few images might not be readable, we'll just skip them.

In [34]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except (IOError, ValueError) as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling /notMNIST_large/A.pickle.
/notMNIST_large/A
Could not read: /notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: /notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: /notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.12824987
Standard deviation: 0.4431207
Pickling /notMNIST_large/B.pickle.
/notMNIST_large/B
Could not read: /notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Full dataset tensor: (52911, 28, 28)
Mean: -0.007563038
Standard deviation: 0.45449144
Pickling /notMNIST_large/C.pickle.
/notMNIST_large/C
Full dataset tensor: (52912, 28, 28)
Mean

In [0]:
#Problem 2
#Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

Problem 2
Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

In [153]:
%matplotlib inline
import random
import hashlib
import matplotlib.image as mpimg


def disp_sample(folders, sapmles):
  for folder in folders:
    print(folder)
    image_files = os.listdir(folder)
    image_sample = random.sample(image_files, sapmles)
    #print(image_sample)
    for image in image_sample:
      image_file = os.path.join(folder, image)
      #print(image_file)
      i = Image(filename = image_file)
      display(i)
      #plt.show(i)
disp_sample(train_folders,2)
disp_sample(test_folders,2)    
    
  #plt.colorbar()

  #print(os.listdir(d))
  #f = os.path.join(d, os.listdir(d)[0])
  #print(f)
  #plt.imshow(mpimg.imread(f))
  #print(f)
  #image = Image(os.path.join(train_folders[0], f)) img=mpimg.imread('stinkbug.png')
  #plt.show(img)
  
#img = Image(os.path.join(train_folders[0], os.listdir(train_folders[0])[0]))
#display(img)

/notMNIST_large/A


/notMNIST_large/B


/notMNIST_large/C


/notMNIST_large/D


/notMNIST_large/E


...

/notMNIST_small/F


/notMNIST_small/G


/notMNIST_small/H


/notMNIST_small/I


/notMNIST_small/J
